## Challenge

The challenge provides $n$, $g$ and $g_m$. 

- $n$ is made up of large primes and we are given the prime factorisation
- $g\equiv g'^{\prod_i p_i-1} \mod n$, where $g'$ is an element in $\mathbb{Z}/n\mathbb{Z}^\times$ guaranteed to have an order that's a divisor of $n$ (order isn't divisible by some small prime). This means $g$ has order a divisor $A = \prod_i p_i^{w_i-1}$.
- $g_m \equiv g^m \mod n$ where $m$ is a random integer smaller than $A$.

We are to recover $m$.

## Solution

Consider a _simpler_ problem $A$: Given $g \equiv g'^{p-1} \mod n$ and $g_m \equiv g^m \mod n$, with $n = p^w$, $p$ be a large prime, $g'$ a generator and $1 < m < p^{w-1}$. Find $m$.

***
_Lemma 1:_

Given $g = g'^{\phi(p^k)} = g'^{(p-1)p^{k-1}}$ and $g_m = g^m$, where $k < w$ and $g'$ a generator of $\mathbb{Z}/p^w\mathbb{Z}^\times$. Then,

$$
\begin{align*}
a &= (g - 1)/{p^k} \\
b &= (g_m - 1)/{p^k} \\
m &= b a^{-1} \mod \; p^k
\end{align*}
$$

We shall refer to this operation as $m = \text{DLPGP}(g, g_m, p, k)$.

_Proof:_

We shall work in $\mathbb{Z}/p^{w'}\mathbb{Z}^\times, \; w' = \min(w, 2k)$

$g^m \equiv 1 + r_mp^k \mod p^{w'}$ where $0 \le r_m < p^k$ due to euler's theorem. Furthermore, $g^{m_1}g^{m_2} \equiv 1 + (r_{m_1} + r_{m_2})p^k \mod p^{w'}$. The map $\mu: m \rightarrow r_m$ is hence an isomorphism from the subgroup generated by $g$ and the subgroup of $\mathbb{Z}/p^k\mathbb{Z}$ generated by $r_1$. We define $\mu = \mu_0 \mu_1$ where $\mu_0: m \rightarrow g^m$ and $\mu_1(g) = (g-1)/p^k$. We have $m = \mu_1(g^m)\mu_1(g)^{-1}$.

***

Back to the problem $A$, we have $m = m_0 + pm_1, \; m_0 = \text{DLPGP}(g, g_m, p, 1)$, and so we'd have to find $m_1$ given $g^{pm_1} = g_m g^{-m_0}$. Notice that $m_1 = (\text{DLPGP}(g^p, g^{pm_1}, p, 2) \mod p) + p m_2$? We can recursively find the value of $m_i$ until $k = w$, where we would have finally recovered $m$! (Note: I took `DLPGP(...) mod p` for ease of implementation).

I refer to the operation of solving problem $A$ as $m = \text{DLP}(g, g_m, p, w)$.

Now consider the original problem in the field $\mathbb{Z}/n\mathbb{Z}^\times$ that's not a prime power. Reducing into $\mathbb{Z}/p_i^{w_i}\mathbb{Z}^\times$ we have, $g_m \equiv g'^{m (p_i - 1) \prod_{j!=i} {p_j - 1}} \equiv g_i^{m (p_i - 1)}\mod p_i^{w_i}$, $g_i = g'^{\prod_{j!=i} {p_j - 1}}$. We hence have $m \equiv \text{DLP}(g_i^{p_i - 1} \equiv g \mod p_i^{w_i}, g_m \mod p_i^{w_i}, p_i, w_i) \mod p_i^{w_i - 1}$. We can hence find $m \mod p_i^{w_i - 1}$ for all $(p_i, w_i)$ and perform [CRT](https://en.wikipedia.org/wiki/Chinese_remainder_theorem) to get $m$!

In [11]:
from params import g, gm, n
from functools import reduce
from hashlib import sha256

def product(l):
    return reduce(lambda a, b: a*b, l)


def dlp_gp(g, gm, p, k):

    """
    Returns m%p^k given gm = g^m = (g'^(p-1)) ^p^(k-1)*m mod p^k
    """

    gr = int(g - 1) // p**k
    gmr = int(gm - 1) // p**k
    return int((gmr * pow(gr,-1,p**k)) % p**k)


def dlp(g, gm, p, w, _d=1):
    
    """
    Returns m%p^(w-1) given gm = g^m = (g'^(p-1)) ^ m mod n
    """
    
    if _d == w: return 0
    
    # m = m1 + p^k*m', g^pm' = gpm
    m1 = dlp_gp(g, gm, p, _d) % p
    gpm = gm * Zmod(p**w)(g)**(-m1)
    gp = pow(g, p, p**w)
    
    return m1 + p*dlp(gp, gpm, p, w, _d+1)


primes, power = n
n = product(p**w for p,w in zip(primes, power))
m = [[dlp(g%p**w, gm%p**w, p, w), p**(w-1)] for p,w in zip(primes, power)]
m = int(CRT_list([x for x,_ in m], [y for _,y in m]))

print("SEE{%s}" % sha256(m.to_bytes((m.bit_length()+7)//8, "little")).hexdigest())

SEE{ca66f51d61e23518c23994777ab1ad2f1c7c4be09ed3d56b91108cf0666d49d1}
